# Tutorial for Small FOV Instruments

In this tutorial we combine our skymaps with galaxy catalogs to get a list of galaxies for individual pointings. A note is made that this is only possible with 3D skymaps which are provided for combact binary merger candidate events.

We begin by importing the necessary packages as done previously. We will also download the 2MASS Redshift Survey galaxy catalog using VizieR.

In [1]:
import healpy as hp # for working with HEALPix files
import numpy as np # needed for vector operations
from matplotlib import pyplot as plt # plotting skymaps
from scipy.stats import norm # probability functions

In [2]:
from astropy.utils.data import download_file
url = ('https://dcc.ligo.org/public/0146/G1701985/001/LALInference_v2.fits.gz')
# This is the publication LALInference localization
filename = download_file(url, cache=True)

We read in the probability, distmu, distsigma, and distnorm.

In [5]:
prob, distmu, distsigma, distnorm = hp.read_map(filename, field=range(4))

NSIDE = 1024
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING
Ordering converted to RING
Ordering converted to RING
Ordering converted to RING


In [6]:
npix = len(prob)
nside = hp.npix2nside(npix)
print(npix, nside)

(12582912, 1024)


In [14]:
# Area per pixel in steradians
pixarea = hp.nside2pixarea(nside)

In [7]:
from astroquery.vizier import Vizier
Vizier.ROW_LIMI = -1
cat, = Vizier.get_catalogs('J/ApJS/199/26/table3') # Downloading the 2MRS Galaxy Catalog

According to Tully(2015), the 2MRS luminosity function is well fit by a Schechter function with a cutoff absolute magnitude of $M_k^* = -23.55$ and a power-law index of $\alpha_K = -1$. We find the maximum absolute magnitude $M_k^{\text{max}}$ for a completeness fraction of 0.5.

In [35]:
from scipy.special import gammaincinv
completeness = 0.5
alpha = -1.0
MK_star = -23.55
MK_max = MK_star + 2.5*np.log10(gammaincinv(alpha + 2, completeness))
MK_max

-23.947936347387156

Now, we select only galaxies with positive redshifts and absolute magnitudes greater than $M_k^{\text{max}}$.

In [36]:
from astropy.cosmology import WMAP9 as cosmo
from astropy.table import Column
import astropy.units as u
import astropy.constants as c
z = (u.Quantity(cat['cz'])/c.c).to(u.dimensionless_unscaled)
MK = cat['Ktmag']-cosmo.distmod(z)
keep = (z > 0) & (MK < MK_max)
cat = cat[keep]
z = z[keep]

Now, we calculate the luminosity distance and HEALPix index of each galaxy.

In [37]:
r = cosmo.luminosity_distance(z).to('Mpc').value
theta = 0.5*np.pi - cat['DEJ2000'].to('rad').value
phi = cat['RAJ2000'].to('rad').value
ipix = hp.ang2pix(nside, theta, phi)

We find the probability density per unit volume at the position of each galaxy.

In [38]:
dp_dV = prob[ipix] * distnorm[ipix] * norm(distmu[ipix], distsigma[ipix]).pdf(r)/pixarea

Finally, we sort the galaxies by descending probability density and take the top 50.

In [39]:
top50 = cat[np.flipud(np.argsort(dp_dV))][:50]
top50['RAJ2000', 'DEJ2000', 'Ktmag']

RAJ2000,DEJ2000,Ktmag
deg,deg,mag
float64,float64,float32
189.99789,-11.62307,4.944
201.36565,-43.01871,3.901
204.25383,-29.86576,4.595
187.44499,8.00041,5.388
186.35022,18.19108,6.134
186.26575,12.88696,6.208
187.70593,12.39110,5.804
190.91670,11.55261,5.730
